In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Double DQN and Game Generalization -- Vizuara

## 1. Why Does This Matter?

In the previous notebook, we trained a DQN agent that learned to play Pong from raw pixels. The algorithm works remarkably well -- but it has a hidden flaw that we have not addressed yet.

Standard DQN systematically **overestimates** Q-values. The $\max$ operator in the target computation is biased upward because the same network both selects and evaluates the best action. Think of it this way: if you ask someone to pick the best option and also judge how good that option is, they will tend to be overconfident.

This overestimation does not always prevent learning, but it can cause instability and suboptimal policies, especially in environments with noisy rewards.

In 2015, Hado van Hasselt, Arthur Guez, and David Silver published "Deep Reinforcement Learning with Double Q-Learning," which introduced **Double DQN** -- a simple but elegant fix. Instead of using the same network for selection and evaluation, Double DQN uses the online network to **select** the best action and the target network to **evaluate** its value.

In this notebook, we will implement Double DQN, demonstrate the overestimation problem, and examine how DQN generalizes across multiple Atari games.

---

## 2. Building Intuition

### The overestimation problem

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random

torch.manual_seed(42)
np.random.seed(42)

# Demo: max of noisy estimates is biased upward
# True Q-values for 4 actions
true_q = np.array([1.0, 1.0, 1.0, 1.0])  # All actions are equally good

# But our estimates have noise
n_trials = 10000
max_estimates = []
double_estimates = []

for _ in range(n_trials):
    # Noisy Q-value estimates (like a neural network with errors)
    noise = np.random.normal(0, 1.0, size=4)
    estimated_q = true_q + noise

    # Standard DQN: max of noisy estimates
    max_estimates.append(np.max(estimated_q))

    # Double DQN: select with one noise sample, evaluate with another
    noise2 = np.random.normal(0, 1.0, size=4)
    estimated_q2 = true_q + noise2
    best_action = np.argmax(estimated_q)  # Select with first estimate
    double_estimates.append(estimated_q2[best_action])  # Evaluate with second

print(f"True max Q-value:     {np.max(true_q):.3f}")
print(f"Standard DQN (biased): {np.mean(max_estimates):.3f} (+{np.mean(max_estimates) - np.max(true_q):.3f} overestimation)")
print(f"Double DQN (unbiased): {np.mean(double_estimates):.3f} (+{np.mean(double_estimates) - np.max(true_q):.3f})")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].hist(max_estimates, bins=50, color='#e74c3c', alpha=0.7, label='Standard DQN')
axes[0].axvline(x=np.max(true_q), color='black', linestyle='--', linewidth=2, label='True value')
axes[0].axvline(x=np.mean(max_estimates), color='#e74c3c', linestyle='-', linewidth=2, label=f'Mean: {np.mean(max_estimates):.2f}')
axes[0].set_title('Standard DQN: Overestimates!', fontweight='bold')
axes[0].set_xlabel('Estimated max Q-value')
axes[0].legend()

axes[1].hist(double_estimates, bins=50, color='#2ecc71', alpha=0.7, label='Double DQN')
axes[1].axvline(x=np.max(true_q), color='black', linestyle='--', linewidth=2, label='True value')
axes[1].axvline(x=np.mean(double_estimates), color='#2ecc71', linestyle='-', linewidth=2, label=f'Mean: {np.mean(double_estimates):.2f}')
axes[1].set_title('Double DQN: Much closer!', fontweight='bold')
axes[1].set_xlabel('Estimated max Q-value')
axes[1].legend()

plt.suptitle('The Overestimation Problem in Q-Learning', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Why overestimation matters

In [ ]:
# Overestimation compounds across multiple steps
gamma = 0.99
n_steps = 100

# Simulate cumulative overestimation
overest_per_step = 0.5  # Average overestimation per step

cumulative_standard = []
cumulative_double = []

for t in range(n_steps):
    # Standard DQN: overestimation accumulates with discount
    cum_standard = sum(overest_per_step * gamma**i for i in range(t+1))
    cumulative_standard.append(cum_standard)

    # Double DQN: much smaller overestimation
    cum_double = sum(0.05 * gamma**i for i in range(t+1))
    cumulative_double.append(cum_double)

plt.figure(figsize=(10, 5))
plt.plot(cumulative_standard, color='#e74c3c', linewidth=2, label='Standard DQN')
plt.plot(cumulative_double, color='#2ecc71', linewidth=2, label='Double DQN')
plt.xlabel('Planning Horizon (steps)', fontsize=12)
plt.ylabel('Cumulative Overestimation', fontsize=12)
plt.title('Overestimation Compounds Over Time', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"After {n_steps} steps:")
print(f"  Standard DQN overestimation: {cumulative_standard[-1]:.2f}")
print(f"  Double DQN overestimation:   {cumulative_double[-1]:.2f}")

---

## 3. The Mathematics

### Standard DQN target

$$y_{\text{DQN}} = r + \gamma \max_{a'} Q(s', a'; \theta^-)$$

The problem: $\max_{a'}$ both **selects** and **evaluates** using the target network. If the target network overestimates any action's value, $\max$ will always pick that overestimation.

### Double DQN target

$$y_{\text{DDQN}} = r + \gamma Q\left(s', \arg\max_{a'} Q(s', a'; \theta),\; \theta^-\right)$$

The fix: use the **online network** $\theta$ to select the best action, but the **target network** $\theta^-$ to evaluate it. This decouples selection from evaluation.

In [ ]:
# Side-by-side comparison
print("=" * 60)
print("Standard DQN vs Double DQN -- The Key Difference")
print("=" * 60)
print()
print("Standard DQN target:")
print("  1. Ask TARGET network: what is max Q(s', a')?")
print("     -> Target both SELECTS and EVALUATES the best action")
print("     -> Biased upward because max of noisy estimates > true max")
print()
print("Double DQN target:")
print("  1. Ask ONLINE network: which action a* is best?")
print("     -> a* = argmax Q(s', a'; theta)")
print("  2. Ask TARGET network: what is Q(s', a*)?")
print("     -> Evaluate a* using target network theta^-")
print("     -> Selection and evaluation are independent")
print()
print("The change is literally ONE line of code!")

In [ ]:
# Worked example
print("\n--- Worked Example ---")
print()

# Online network predictions for next state
q_online_next = torch.tensor([2.5, 3.1, 1.8, 2.9])
# Target network predictions for next state
q_target_next = torch.tensor([2.3, 2.8, 1.9, 3.2])
r = 1.0
gamma = 0.99

# Standard DQN
max_target_q = q_target_next.max().item()
y_dqn = r + gamma * max_target_q
print(f"Standard DQN:")
print(f"  Target net Q-values: {q_target_next.numpy()}")
print(f"  max Q(s', a'; theta^-) = {max_target_q:.2f} (action {q_target_next.argmax().item()})")
print(f"  y_DQN = {r} + {gamma} * {max_target_q:.2f} = {y_dqn:.4f}")

# Double DQN
best_action_online = q_online_next.argmax().item()
eval_target = q_target_next[best_action_online].item()
y_ddqn = r + gamma * eval_target
print(f"\nDouble DQN:")
print(f"  Online net Q-values:  {q_online_next.numpy()}")
print(f"  Best action (online): {best_action_online} (Q = {q_online_next[best_action_online]:.2f})")
print(f"  Target net evaluates action {best_action_online}: Q = {eval_target:.2f}")
print(f"  y_DDQN = {r} + {gamma} * {eval_target:.2f} = {y_ddqn:.4f}")

print(f"\nDifference: {y_dqn - y_ddqn:.4f} (DQN target is higher)")
print("This small difference per step accumulates over training!")

---

## 4. Let's Build It -- Component by Component

### Step 1: Standard vs Double DQN loss

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class DQN(nn.Module):
    def __init__(self, n_actions):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4), nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU()
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 7 * 7, 512), nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x.view(x.size(0), -1))


def compute_loss_standard(online_net, target_net, states, actions, rewards, next_states, dones, gamma=0.99):
    """Standard DQN loss."""
    q_values = online_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)

    with torch.no_grad():
        # Standard: max from target network
        max_next_q = target_net(next_states).max(1)[0]
        target = rewards + gamma * max_next_q * (~dones).float()

    return F.mse_loss(q_values, target)


def compute_loss_double(online_net, target_net, states, actions, rewards, next_states, dones, gamma=0.99):
    """Double DQN loss -- the only change is in target computation."""
    q_values = online_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)

    with torch.no_grad():
        # Double DQN: SELECT with online, EVALUATE with target
        best_actions = online_net(next_states).argmax(1)  # <-- online selects
        next_q = target_net(next_states).gather(1, best_actions.unsqueeze(1)).squeeze(1)  # <-- target evaluates
        target = rewards + gamma * next_q * (~dones).float()

    return F.mse_loss(q_values, target)


# Compare the two on identical data
n_actions = 4
online = DQN(n_actions).to(device)
target = DQN(n_actions).to(device)
target.load_state_dict(online.state_dict())

# Fake batch
batch_size = 32
s = torch.randn(batch_size, 4, 84, 84).to(device)
a = torch.randint(0, n_actions, (batch_size,)).to(device)
r = torch.randn(batch_size).to(device)
ns = torch.randn(batch_size, 4, 84, 84).to(device)
d = torch.zeros(batch_size, dtype=torch.bool).to(device)

loss_std = compute_loss_standard(online, target, s, a, r, ns, d)
loss_dbl = compute_loss_double(online, target, s, a, r, ns, d)

print(f"Standard DQN loss: {loss_std.item():.4f}")
print(f"Double DQN loss:   {loss_dbl.item():.4f}")
print(f"\nWith identical weights, both losses are similar.")
print("The difference emerges during training as networks diverge.")

### Step 2: Tracking Q-value estimates during training

In [ ]:
from collections import deque

class QValueTracker:
    """Track Q-value estimates over training to detect overestimation."""
    def __init__(self, n_eval_states=100):
        self.eval_states = None
        self.n_eval_states = n_eval_states
        self.q_history = []

    def set_eval_states(self, replay_buffer):
        """Sample fixed evaluation states from the buffer."""
        if len(replay_buffer) >= self.n_eval_states:
            batch = replay_buffer.sample(self.n_eval_states)
            self.eval_states = batch[0]

    def record(self, online_net):
        """Record average Q-value on evaluation states."""
        if self.eval_states is None:
            return
        with torch.no_grad():
            q = online_net(self.eval_states.to(device))
            avg_q = q.max(1)[0].mean().item()
            self.q_history.append(avg_q)

tracker_standard = QValueTracker()
tracker_double = QValueTracker()
print("Q-value tracker ready. Will use during training comparison.")

---

## 5. Your Turn

### TODO 1: Implement Dueling DQN architecture

Double DQN changes the loss computation. **Dueling DQN** changes the architecture. Instead of outputting Q-values directly, it separates the value function $V(s)$ and advantage function $A(s,a)$:

$$Q(s, a) = V(s) + A(s, a) - \frac{1}{|A|}\sum_{a'} A(s, a')$$

In [ ]:
# TODO: Implement the Dueling DQN architecture
# The scaffolding is provided -- fill in the stream definitions and forward pass.

class DuelingDQN(nn.Module):
    def __init__(self, n_actions):
        super().__init__()
        # Same convolutional backbone
        self.conv = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4), nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU()
        )

        # Value stream: 3136 -> 512 -> 1 (single state value)
        self.value_stream = nn.Sequential(
            nn.Linear(64 * 7 * 7, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

        # Advantage stream: 3136 -> 512 -> n_actions (per-action advantage)
        self.advantage_stream = nn.Sequential(
            nn.Linear(64 * 7 * 7, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)

        value = self.value_stream(x)            # (batch, 1)
        advantage = self.advantage_stream(x)    # (batch, n_actions)
        # Combine: Q = V + A - mean(A)
        q = value + advantage - advantage.mean(dim=1, keepdim=True)
        return q

# Test: DuelingDQN should have the same input/output shape as DQN
dueling = DuelingDQN(4).to(device)
test = torch.randn(1, 4, 84, 84).to(device)
print(f"Dueling DQN output: {dueling(test).shape}")  # Should be [1, 4]

# Compare parameter counts
dqn_params = sum(p.numel() for p in DQN(4).parameters())
dueling_params = sum(p.numel() for p in dueling.parameters())
print(f"Standard DQN params: {dqn_params:,}")
print(f"Dueling DQN params:  {dueling_params:,}")
print(f"Dueling has two separate FC streams -> more parameters")

### TODO 2: Compare overestimation empirically

In [ ]:
# TODO: Train both standard and double DQN on the substitute environment
# and compare their Q-value estimates over time.
#
# 1. Train standard DQN for 200 episodes, recording avg Q-values every 10 episodes
# 2. Train Double DQN for 200 episodes, recording avg Q-values every 10 episodes
# 3. Plot both Q-value curves on the same graph
# 4. The standard DQN curve should be consistently ABOVE the Double DQN curve
#    (overestimation)
#
# YOUR CODE HERE

---

## 6. Putting It All Together

### DQN performance across Atari games

In [ ]:
# Reproduce the key results from the DQN Nature paper
# Data from Mnih et al. (2015), Table 1

games_data = {
    'Breakout':           {'human': 31.8,  'dqn': 401.2, 'category': 'superhuman'},
    'Pong':               {'human': 9.3,   'dqn': 18.9,  'category': 'superhuman'},
    'Space Invaders':     {'human': 1652,  'dqn': 1976,  'category': 'superhuman'},
    'Boxing':             {'human': 4.3,   'dqn': 71.8,  'category': 'superhuman'},
    'Enduro':             {'human': 309.6, 'dqn': 475.6, 'category': 'superhuman'},
    'Beam Rider':         {'human': 7456,  'dqn': 8627,  'category': 'superhuman'},
    'Seaquest':           {'human': 20182, 'dqn': 5286,  'category': 'below'},
    "Montezuma's Rev.":   {'human': 4367,  'dqn': 0,     'category': 'fails'},
    'Private Eye':        {'human': 69571, 'dqn': 146,   'category': 'fails'},
}

# Compute DQN as % of human
games = list(games_data.keys())
pct_human = []
colors = []
for g in games:
    d = games_data[g]
    if d['human'] > 0:
        pct = (d['dqn'] / d['human']) * 100
    else:
        pct = 100
    pct_human.append(min(pct, 1500))  # Cap for visualization
    if d['category'] == 'superhuman':
        colors.append('#2ecc71')
    elif d['category'] == 'below':
        colors.append('#f39c12')
    else:
        colors.append('#e74c3c')

# Sort by performance
sorted_indices = np.argsort(pct_human)[::-1]
games_sorted = [games[i] for i in sorted_indices]
pct_sorted = [pct_human[i] for i in sorted_indices]
colors_sorted = [colors[i] for i in sorted_indices]

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(range(len(games_sorted)), pct_sorted, color=colors_sorted, edgecolor='white', height=0.7)
ax.axvline(x=100, color='black', linestyle='--', linewidth=2, label='Human level (100%)')
ax.set_yticks(range(len(games_sorted)))
ax.set_yticklabels(games_sorted, fontsize=11)
ax.set_xlabel('DQN Score as % of Human Performance', fontsize=12)
ax.set_title('DQN Performance Across Atari Games (Mnih et al., 2015)', fontsize=14, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(axis='x', alpha=0.3)

# Add percentage labels
for i, (bar, pct) in enumerate(zip(bars, pct_sorted)):
    ax.text(bar.get_width() + 10, bar.get_y() + bar.get_height()/2,
            f'{pct:.0f}%', va='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("  Superhuman (green): Games requiring pattern recognition and reactive play")
print("  Below human (orange): Games with complex reward structures")
print("  Fails (red): Games requiring long-term memory and sparse rewards")

In [ ]:
# Why DQN fails on some games
print("=" * 60)
print("WHY DQN FAILS ON MONTEZUMA'S REVENGE")
print("=" * 60)
print("""
Montezuma's Revenge requires:
  1. Long sequences of actions before any reward (sparse rewards)
  2. Remembering which rooms were visited (long-term memory)
  3. Picking up specific keys to open specific doors (planning)

DQN's epsilon-greedy exploration is purely random.
The probability of randomly discovering the correct sequence
of 100+ actions to reach the first reward is astronomically low.

This led to research on:
  - Curiosity-driven exploration (intrinsic motivation)
  - Hierarchical RL (options, sub-goals)
  - Go-Explore (archive-based exploration)
  - Return-conditioned decision transformers
""")

---

## 7. Training and Results

### The DQN family -- improvements over original

In [ ]:
# Compare the evolution of DQN improvements
improvements = {
    'DQN (2013)':                   {'score_pct': 121, 'year': 2013},
    'Double DQN (2015)':            {'score_pct': 149, 'year': 2015},
    'Prioritized Replay (2015)':    {'score_pct': 192, 'year': 2015},
    'Dueling DQN (2016)':           {'score_pct': 175, 'year': 2016},
    'A3C (2016)':                   {'score_pct': 116, 'year': 2016},
    'NoisyNet (2017)':              {'score_pct': 218, 'year': 2017},
    'Distributional (2017)':        {'score_pct': 315, 'year': 2017},
    'Rainbow (2017)':               {'score_pct': 427, 'year': 2017},
}

names = list(improvements.keys())
scores = [improvements[n]['score_pct'] for n in names]
years = [improvements[n]['year'] for n in names]

fig, ax = plt.subplots(figsize=(12, 6))
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(names)))
bars = ax.bar(range(len(names)), scores, color=colors, edgecolor='white', width=0.7)
ax.axhline(y=100, color='red', linestyle='--', linewidth=2, label='Human level')
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=45, ha='right', fontsize=10)
ax.set_ylabel('Median Score (% of Human)', fontsize=12)
ax.set_title('Evolution of DQN: Each Improvement Builds on the Last', fontsize=14, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(axis='y', alpha=0.3)

for bar, score in zip(bars, scores):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 5,
            f'{score}%', ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

print("\nRainbow (2017) combines ALL improvements:")
print("  1. Double DQN (reduces overestimation)")
print("  2. Prioritized replay (focus on important transitions)")
print("  3. Dueling architecture (separate value and advantage)")
print("  4. Multi-step returns (n-step bootstrapping)")
print("  5. Distributional RL (predict return distribution)")
print("  6. NoisyNets (learned exploration)")

---

## 8. Final Output

In [ ]:
print("=" * 60)
print("COURSE SUMMARY: Building DQN Atari Agents")
print("=" * 60)
print(f"""
Across these 4 notebooks, we built DQN from scratch:

  Notebook 1: The DQN Architecture
    - Function approximation: Q-table -> neural network
    - CNN processes 4 stacked 84x84 grayscale frames
    - Outputs Q-values for every possible action

  Notebook 2: Experience Replay and Target Networks
    - Replay buffer breaks temporal correlations
    - Target network provides stable training targets
    - Both are essential for training stability

  Notebook 3: Complete Training Pipeline
    - Epsilon-greedy with annealing
    - Full training loop on Pong
    - Agent learns from raw pixels to win games

  Notebook 4: Double DQN and Generalization
    - Standard DQN overestimates Q-values
    - Double DQN: separate selection and evaluation
    - DQN excels at reactive play, struggles with planning
    - Rainbow combines all improvements

Historical Significance:
  - DQN (2013) was the first successful deep RL agent
  - Same architecture worked on 49 different games
  - Led to Google's acquisition of DeepMind for ~\\$500M
  - Opened the door to AlphaGo, robotics, and modern RLHF

These same principles -- function approximation, replay buffers,
target networks -- are used in virtually every modern deep RL system.
""")

---

## 9. Reflection and Next Steps

**What we built:**
- Double DQN loss computation (one line change from standard DQN)
- Visualization of the overestimation problem
- Analysis of DQN performance across Atari games
- Survey of the DQN improvement family (Rainbow)

**Key insights:**
1. The $\max$ operator causes systematic overestimation of Q-values
2. Double DQN decouples action selection from value evaluation
3. DQN excels at reactive pattern recognition but fails at long-horizon planning
4. Rainbow combines 6 improvements for state-of-the-art performance

**The bigger picture:**
- DQN showed that a single neural network can learn directly from pixels
- The three core ideas (CNNs for function approximation, experience replay, target networks) remain foundational in modern deep RL
- The limitations of DQN (sparse rewards, planning) motivated entire subfields: curiosity-driven exploration, hierarchical RL, and model-based RL

**Where to go next:**
- Policy gradient methods (REINFORCE, PPO) -- a fundamentally different approach to RL
- Actor-critic methods that combine value-based and policy-based learning
- Model-based RL where the agent learns a world model and plans inside it